<a href="https://colab.research.google.com/github/Manish-k723/100-days-of-ML/blob/main/Day_28_Column_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv')

In [4]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
40,49,Female,102.0,Mild,Delhi,No
58,23,Male,98.0,Strong,Mumbai,Yes
66,51,Male,104.0,Mild,Kolkata,No
18,64,Female,98.0,Mild,Bangalore,Yes
67,65,Male,99.0,Mild,Bangalore,No


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [8]:
df.describe()

,age,fever
count,100.000000,90.000000
mean,44.220000,100.844444
std,24.878931,2.054926
min,5.000000,98.000000
25%,20.000000,99.000000
50%,45.000000,101.000000
75%,66.500000,102.750000
max,84.000000,104.000000


In [6]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop('has_covid', axis = 1), df['has_covid'], test_size = 0.2, random_state = 42)

x_train.shape, x_test.shape

((80, 5), (20, 5))

# **Normal Way of doing**
##`#aam_jindgi`

In [20]:
si = SimpleImputer()

x_train_si = si.fit_transform(x_train[['fever']])

x_test_si = si.transform(x_test[['fever']])

In [23]:
df.cough.unique()

array(['Mild', 'Strong'], dtype=object)

In [ ]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

x_train_oe = oe.fit_transform(x_train[['cough']])

x_test_oe = oe.transform(x_test[['cough']])

x_train_oe

In [26]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [ ]:
ohe = OneHotEncoder(drop = 'first', sparse = False)

x_train_ohe = ohe.fit_transform(x_train[['gender','city']])

x_test_ohe = ohe.transform(x_test[['gender','city']])

x_train_ohe

In [31]:
x_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
...,...,...,...,...,...
60,24,Female,102.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
14,51,Male,104.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata


In [42]:
x_train_age = x_train.drop(columns = ['gender','fever',	'cough',	'city']).values

x_test_age = x_test.drop(columns = ['gender','fever',	'cough',	'city']).values

x_train_age.shape

(80, 1)

In [43]:
x_train_trans = np.concatenate((x_train_age,x_train_si, x_train_ohe,x_train_oe), axis = 1)

x_test_trans = np.concatenate((x_test_age,x_test_si, x_test_ohe,x_test_oe), axis = 1)

In [44]:
x_train_trans.shape

(80, 7)

In [46]:
df.city.unique

<bound method Series.unique of 0       Kolkata
1         Delhi
2         Delhi
3       Kolkata
4        Mumbai
        ...    
95    Bangalore
96      Kolkata
97    Bangalore
98       Mumbai
99      Kolkata
Name: city, Length: 100, dtype: object>

# **Smart Way...!**
##`#mentos_jindgi`

In [50]:
from sklearn.compose import ColumnTransformer

In [51]:
transformer = ColumnTransformer(transformers = [
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories = [['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse = False),['gender','city'])
], remainder = 'passthrough')

In [53]:
transformer.fit_transform(x_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[101.,   0.,   0.,   0.,   0.,   1.,  81.],
       [100.,   0.,   0.,   0.,   1.,   0.,   5.],
       [100.,   0.,   0.,   0.,   1.,   0.,  19.],
       [100.,   0.,   1.,   1.,   0.,   0.,  27.],
       [103.,   0.,   0.,   1.,   0.,   0.,  73.],
       [103.,   1.,   1.,   0.,   1.,   0.,  70.],
       [102.,   0.,   0.,   1.,   0.,   0.,  49.],
       [101.,   1.,   0.,   0.,   1.,   0.,  51.],
       [101.,   0.,   0.,   1.,   0.,   0.,  64.],
       [101.,   0.,   0.,   0.,   1.,   0.,  83.],
       [ 98.,   0.,   0.,   0.,   0.,   1.,  65.],
       [104.,   0.,   0.,   0.,   0.,   0.,  18.],
       [103.,   0.,   0.,   0.,   0.,   0.,  16.],
       [104.,   0.,   1.,   0.,   1.,   0.,  16.],
       [100.,   0.,   1.,   0.,   1.,   0.,  27.],
       [101.,   0.,   0.,   0.,   0.,   0.,  84.],
       [104.,   0.,   1.,   0.,   1.,   0.,  51.],
       [102.,   0.,   0.,   0.,   0.,   0.,  69.],
       [102.,   1.,   0.,   0.,   0.,   0.,  82.],
       [103.,   0.,   0.,   0.,

In [55]:
transformer.transform(x_test)

array([[104.,   0.,   0.,   0.,   1.,   0.,  17.],
       [ 98.,   0.,   1.,   1.,   0.,   0.,  83.],
       [101.,   1.,   0.,   1.,   0.,   0.,  68.],
       [ 99.,   0.,   1.,   0.,   0.,   0.,  72.],
       [102.,   1.,   1.,   1.,   0.,   0.,  20.],
       [103.,   0.,   0.,   0.,   1.,   0.,  50.],
       [ 98.,   1.,   0.,   0.,   1.,   0.,  71.],
       [ 99.,   0.,   0.,   0.,   0.,   1.,  14.],
       [101.,   0.,   0.,   1.,   0.,   0.,  75.],
       [103.,   0.,   1.,   0.,   1.,   0.,  60.],
       [ 98.,   0.,   0.,   0.,   0.,   0.,  64.],
       [101.,   0.,   1.,   1.,   0.,   0.,  15.],
       [ 98.,   1.,   1.,   0.,   1.,   0.,  34.],
       [ 98.,   0.,   0.,   0.,   1.,   0.,  26.],
       [ 99.,   1.,   0.,   1.,   0.,   0.,  59.],
       [101.,   0.,   0.,   0.,   0.,   1.,  65.],
       [100.,   0.,   1.,   0.,   0.,   0.,  80.],
       [101.,   0.,   0.,   0.,   1.,   0.,   8.],
       [ 99.,   1.,   0.,   0.,   1.,   0.,  25.],
       [103.,   0.,   1.,   0.,